# Practice Problem for CEE 201 Fall 2023 Project

#### Reading data
The following piece of code reads the data from the text file. Please adjust the location of the folder in accordance to your machine.

In [7]:
import numpy as np

with open("small_test.txt") as file:
    data = [line.rstrip() for line in file]

streets=[]
intersections=[]
for line in data:
    l = line.split()
    streets.append((int(l[0]), int(l[1]), float(l[2])))
    streets.append((int(l[1]), int(l[0]), float(l[2])))
    intersections.append(int(l[0]))
    intersections.append(int(l[1]))
intersections=list(np.unique(intersections))

In [8]:
print("The network has", len(streets), "streets and", len(intersections), "intersections.")

The network has 28 streets and 10 intersections.


#### Solving the practice problem
We will solve the Practice Problem using PuLP, a solver that you should have already basic knowldege of using.

In [9]:
from pulp import *

In [10]:
#Set up the model
model = LpProblem("Shortest_Path_Problem",LpMinimize)
# Define the decision variables
xindx=[]
for street in streets:
    xindx.append((street[0], street[1]))
    
x = LpVariable.dicts("X", xindx,0,None) #setting up x variables and nonnegativity constraints
model+=lpSum([street[2]*x[street[0], street[1]] for street in streets])

In [11]:
#Set up the flow conservation constraints
source=0
terminal=9
for i in intersections:
    if i==source:
        model+=lpSum([x[street[0],street[1]]-x[street[1],street[0]] for street in streets if street[0]==i])==1, "source"
    elif i==terminal:
        model+=lpSum([x[street[0],street[1]]-x[street[1],street[0]] for street in streets if street[0]==i])==-1, "destination"
    else:
        model+=lpSum([x[street[0],street[1]]-x[street[1],street[0]] for street in streets if street[0]==i])==0, "node"+str(i)

In [12]:
model.solve()
print('Status:',LpStatus[model.status])

Status: Optimal


#### Obtaining the solution
Here we print the optimal path and its travel time.

In [13]:
for v in model.variables():
    if v.varValue==1:
        print(v.name, '=', v.varValue)
print('The travel time is', value(model.objective))

X_(0,_1) = 1.0
X_(1,_3) = 1.0
X_(3,_5) = 1.0
X_(5,_7) = 1.0
X_(7,_8) = 1.0
X_(8,_9) = 1.0
The travel time is 6.0
